<a href="https://colab.research.google.com/github/davidharrisnet/genai/blob/main/llama2_7b_chat_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import os

In [7]:
!pip3 install virtualenv
!virtualenv llama_env

created virtual environment CPython3.10.12.final.0-64 in 182ms
  creator CPython3Posix(dest=/content/llama_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.3.1, setuptools==69.0.2, wheel==0.42.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [11]:
os.path.exists("/content/llama_env/bin")

True

In [10]:
sys.path.insert(0,"/content/llama_env/lib/python3.10/site-packages")

In [12]:
os.environ["PATH"] = "/content/llama_env/bin:" + os.environ["PATH"]


In [13]:
os.environ["PATH"]

'/content/llama_env/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin'

In [16]:
!pip install transformers accelerate einops langchain[all] bitsandbytes sentence_transformers  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [17]:
import os
from torch import cuda, bfloat16
from time import time
import transformers
import torch

In [18]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [19]:
model_id = os.path.join("/content","drive","My Drive", "models", "Llama-2-7b-chat-hf", "snapshots", "1")

In [20]:
os.path.exists(model_id)


True

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig

In [22]:
tokenizer = AutoTokenizer.from_pretrained(model_id)


In [23]:
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
print(model.get_memory_footprint())

7067942912


In [25]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [26]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
from langchain.chains import ConversationChain
from langchain import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.conversation.memory import  ConversationSummaryMemory

In [27]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=10000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.7})

In [28]:
prompt_template = """<s>[INST] <<SYS>>
{{ You are an intelligence analyst}}<<SYS>>
###

Previous Conversation:
'''
{history}
'''

{{{input}}}[/INST]

"""
prompt  = PromptTemplate(template=prompt_template, input_variables=['input', 'history'])

In [29]:
#memory = ConversationSummaryMemory(llm=llm)
memory = ConversationBufferWindowMemory(k=10)

chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [30]:
questions = [ "Write me a detailed scenario for a potential Russian invasion of Moldova.",

"Now identify one person in this scenario who would be a potential HUMINT source for the US Intelligence Community and write a short source dossier. Be sure to include their placement and access.",
"Write me three short intelligence information reports that came from that source about various events in the scenario.",
"Add dates and times for the intelligence reports." ]

In [31]:
chain.run(questions[0])

"As an intelligence analyst, I must first emphasize that the following scenario is purely hypothetical and does not reflect any current or credible intelligence on the intentions of the Russian Federation.\n\nScenario:\n\nThe Russian Federation, under the leadership of President Vladimir Putin, has been increasingly aggressive in its foreign policy, particularly in the region of Eastern Europe. In recent months, Moscow has been engaged in a series of military exercises and troop movements near the border of Moldova, a small, former Soviet republic.\n\nIntelligence suggests that Russia has been seeking to exploit internal divisions within Moldova, which has a fragile political landscape and a history of conflict with its larger neighbor. Moscow has been secretly supporting various pro-Russian political groups and militias within Moldova, with the aim of creating a pretext for military intervention.\n\nOn April 10th, 2023, the Moldovan government announces the discovery of a Russian-back

In [32]:
chain.run(questions[1])

'Source Dossier:\n\nName: Alexei Petrov\n\nPlacement: Alexei Petrov is a former Moldovan military officer who defected to Russia in 2018. He currently resides in Moscow and works as a freelance security consultant.\n\nAccess: Alexei has maintained contacts within the Moldovan military and political circles, including former colleagues and associates who are now part of the pro-Russian groups that Moscow is allegedly supporting. He has also established connections with some of the Russian military officials who are involved in the planning and execution of the invasion.\n\nSkills and Expertise: Alexei has extensive knowledge of the Moldovan military and political structures, as well as the dynamics of the region. He also has experience in military planning and tactics, which could be invaluable to the US Intelligence community in understanding the Russian invasion plans.\n\nPersonality: Alexei is a pragmatic individual who is highly motivated by financial gain. He has a history of being

In [33]:
chain.run(questions[2])

'Intelligence Information Report 1:\n\nTitle: Russian Military Buildup Along Moldovan Border Continues\n\nSource: Alexei Petrov\nDate: April 15, 2023\n\nSummary: According to sources within the Russian military, the buildup of Russian troops along the Moldovan border is continuing at an alarming rate. The Russian military is reportedly preparing for a large-scale invasion of Moldova, with the aim of overthrowing the Moldovan government and annexing the country to Russia. The invasion is expected to begin within the next 72 hours.\n\nIntelligence Information Report 2:\n\nTitle: Pro-Russian Elements Within Moldovan Military Planning Coup\n\nSource: Alexei Petrov\nDate: April 17, 2023\n\nSummary: According to Alexei Petrov, pro-Russian elements within the Moldovan military are actively planning a coup against the Moldovan government. The coup is expected to take place within the next 48 hours and will likely involve the overthrow of the government and the installation of a pro-Russian lea

In [34]:
chain.run(questions[3])

'Intelligence Information Report 1:\nApril 15, 2023, 14:00 UTC\n\nTitle: Russian Military Buildup Along Moldovan Border Continues\n\nSource: Alexei Petrov\n\nSummary: According to sources within the Russian military, the buildup of Russian troops along the Moldovan border is continuing at an alarming rate. The Russian military is preparing for a large-scale invasion of Moldova, with the aim of overthrowing the Moldovan government and annexing the country to Russia. The invasion is expected to begin within the next 72 hours.\n\nIntelligence Information Report 2:\nApril 17, 2023, 12:00 UTC\n\nTitle: Pro-Russian Elements Within Moldovan Military Planning Coup\n\nSource: Alexei Petrov\n\nSummary: According to Alexei Petrov, pro-Russian elements within the Moldovan military are actively planning a coup against the Moldovan government. The coup is expected to take place within the next 48 hours and will likely involve the overthrow of the government and the installation of a pro-Russian lead